# DataFrames II: Filtering Data

In [1]:
import pandas as pd

## This Module's Dataset + Memory Optimization
- The `pd.to_datetime` method converts a **Series** to hold datetime values.
- The `format` parameter informs pandas of the format that the times are stored in.
- We pass symbols designating the segments of the string. For example, %m means "month" and %d means day.
- The `dt` attribute reveals an object with many datetime-related attributes and methods.
- The `dt.time` attribute extracts only the time from each value in a datetime **Series**.
- Use the `astype` method to convert the values in a **Series** to another type.
- The `parse_dates` parameter of `read_csv` is an alternate way to parse strings as datetimes.

In [2]:
emp = pd.read_csv("employees.csv")
emp.info() # Vemos uma prévia do dataset, onde podemos já trabalhar o ETL

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   First Name         933 non-null    object 
 1   Gender             855 non-null    object 
 2   Start Date         1000 non-null   object 
 3   Last Login Time    1000 non-null   object 
 4   Salary             1000 non-null   int64  
 5   Bonus %            1000 non-null   float64
 6   Senior Management  933 non-null    object 
 7   Team               957 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 62.6+ KB


In [3]:
# Podemos começar o ETL adequando as variáveis aos seus tipos, para que esteja adequado
# Adequando datas, que obecem formatos variáveis, e agora começaremos a chamar funções da biblioteca, não atribuídas ao dataframe direto
emp["Start Date"] = pd.to_datetime(emp["Start Date"]) # Sempre definir a igualdade quando queremos fazer a visão ou o objeto em si se tornar um valor, a não ser casos que alteram ele direto ou cópias
emp["Start Date"] = pd.to_datetime(emp["Start Date"], format="%m/%d/%Y")# Podemos formatar a data a partir de um campo que já esteja devidamente formatado e identificado como data (tem de estar reconhecível), assim como fazemos no SQL ou PBI
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y") #Podemos também definir a data e alguns outros tipos durante a seleção do arquivo para poupar código e processamento, além da data (que pode ser em lista também, sempre podemos inserir listas), também podemos fazer isso com outros analisando os atributos
emp.tail()

# Adequando horas 
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time # Em python extraímos os valores ponto por ponto, então juntamos atributos e funções maiores para chegar nas menores, assim como fazemos agora. Da função que converteu, puxamos o resultado (date time) e deste puxamos o tempo. Temos de nos organizar deste modo, se queremos extrair um valor menor, primeiro fazemos a operação e então vamos do maior para o menor
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [4]:
# Agora podemos organizar outros tipos
emp["Senior Management"] = emp["Senior Management"].astype(bool) # Sempre analisar quais os tipos adequados, ou seja, analisar o dataset
emp["Gender"] = emp["Gender"].astype("category") # Lembrando que valores que se repetem massivamente ou significativamente serão categorias, então podem ser deste tipo
emp.info() # Poderíamos ainda limpar outros tipos para string e definir menores que o tamanho reservado para objetos por exemplo, mas este é apenas um exemplo


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   First Name         933 non-null    object        
 1   Gender             855 non-null    category      
 2   Start Date         1000 non-null   datetime64[ns]
 3   Last Login Time    1000 non-null   object        
 4   Salary             1000 non-null   int64         
 5   Bonus %            1000 non-null   float64       
 6   Senior Management  1000 non-null   bool          
 7   Team               957 non-null    object        
dtypes: bool(1), category(1), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 49.1+ KB


## Filter A DataFrame  Based On A Condition
- Pandas needs a **Series** of Booleans to perform a filter.
- Pass the Boolean Series inside square brackets after the **DataFrame**.
- We can generate a Boolean Series using a wide variety of operations (equality, inequality, less than, greater than, inclusion, etc)

In [5]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m-%d-%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   First Name         933 non-null    object  
 1   Gender             855 non-null    category
 2   Start Date         1000 non-null   object  
 3   Last Login Time    1000 non-null   object  
 4   Salary             1000 non-null   int64   
 5   Bonus %            1000 non-null   float64 
 6   Senior Management  1000 non-null   bool    
 7   Team               957 non-null    object  
dtypes: bool(1), category(1), float64(1), int64(1), object(4)
memory usage: 49.1+ KB


In [6]:
# Para filtrarmos valores com condições dentro de python, temos de especificar o que queremos e o que não queremos
# Lembrando que o operador de comparação é ==
emp["Gender"] == "Male" # Deste modo, filtramos valores trazendo a igualdade, pois  ele retornará true para cada linha aceita e false para cada uma não aceita

0       True
1       True
2      False
3       True
4       True
       ...  
995    False
996     True
997     True
998     True
999     True
Name: Gender, Length: 1000, dtype: bool

In [7]:
emp[emp["Gender"] == "Male"] # Podemos então filtrar valores, uma vez que passamos uma lista de valores aceitos, ou seja, um conjunto de valores com true que python pode retornar, uma série que ele nativamente bate com os valores e retorna, pois ele faz a comparação e como é uma série, a partir dessa comparação retorna os valores
emp[emp["Team"] == "Finance"] # Outro exemplo
filtro = emp["Team"] == "Finance" # Se for confuso ou para utilizarmos  em códigos onde o usuário define a variável, podemos  definir aquele trecho para uma variável e então utilizá-lo, pois em python é possível, então o usuário define o trecho e inserimos no código (em python podemos fazer com strings e outros tipos também)
emp[filtro]

emp[emp["Senior Management"]].head() # Podemos extrair cabeçalhos e filtrar limites a partir da consulta, assim como vimos, algo maior se torna algo menor
# Para booleanos devemos nos atentar, pois ele vai comparar ambos então se queremos true, ele sempre vai dar certo, logo, como acima, não precisamos dar igualdade. Se queremos false, basta passar false que ele vai comparaar e o que antes era false vira true, afinal, true == true ainda vai ser true e true == false ainda vai ser false e assim por diante

emp["Salary"] > 110000 # Antes de fazer, podemos rodar os testes de true e false para verificar se definimos as condições e o filtro corretamente e se atendem, então podemos ter uma prévia e ver se definimos certo para puxar os valores com true (que definimos)
emp[emp["Salary"] > 110000]
emp[emp["Bonus %"] < 1.5]
emp[emp["Start Date"]  < "1985-01-01"] # Com datas, podemos fazer as comparações passando o formato correto, afinal, ele vai fazer comparações com cada número da data separadamente, mas identificando que é uma data por causa do  campo comparado (não precisamos saber desses detalhes), logo, sabendo o tipo de data podemos passar as comparações normalmente

# Já para trabalhar com hora
import datetime as dt
dt.time(12, 0, 0) # Usaremos para comparar, representado horas, minutos e segundos, pois quando comparamos tempo, por dentro ele converte para este formato
emp[emp["Last Login Time"] < dt.time(12,0,0)] # Representando o tipo convertido, temos de comparar deste modo, python apenas entende assim pois é deste modo que enxerga horas por trás da linguagem(?)


,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
1,Thomas,Male,3/31/1996,06:53:00,61933,4.170,True,NaN
2,Maria,Female,4/23/1993,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,3/4/2005,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1/24/1998,04:47:00,101004,1.389,True,Client Services
5,Dennis,Male,4/18/1987,01:35:00,115163,10.125,False,Legal
...,...,...,...,...,...,...,...,...
994,George,Male,6/21/2013,05:47:00,98874,4.479,True,Marketing
995,Henry,NaN,11/23/2014,06:09:00,132483,16.655,False,Distribution
996,Phillip,Male,1/31/1984,06:30:00,42392,19.675,False,Finance
998,Larry,Male,4/20/2013,04:45:00,60500,11.985,False,Business Development


## Filter with More than One Condition (AND)
- Add the `&` operator in between two Boolean **Series** to filter by multiple conditions.
- We can assign the **Series** to variables to make the syntax more readable.

In [8]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [9]:
# Podemos filtrar vários valores com o operador and &
# Supondo que definimos uma variáavel para o usuário utilizar como filtro, assim como fizemos anteriormente
is_female = emp["Gender"] == "Female"
is_marketing = emp["Team"] == "Marketing"
emp[is_female & is_marketing] # Lembrando sempre da regra do and, que ambas tem de ser verdadeiras

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
43,Marilyn,Female,1980-12-07,03:16:00,73524,5.207,True,Marketing
62,NaN,Female,2007-06-12,05:25:00,58112,19.414,True,Marketing
98,Tina,Female,2016-06-16,07:47:00,100705,16.961,True,Marketing
140,Shirley,Female,1981-02-28,01:23:00,113850,1.854,False,Marketing
158,Norma,Female,1999-02-28,08:45:00,114412,8.756,True,Marketing
201,Kimberly,Female,1997-07-15,05:57:00,36643,7.953,False,Marketing
220,NaN,Female,1991-06-17,12:49:00,71945,5.560,True,Marketing
305,Margaret,Female,1993-02-06,01:05:00,125220,3.733,False,Marketing
319,Jacqueline,Female,1981-11-25,03:01:00,145988,18.243,False,Marketing
331,Evelyn,Female,1983-09-03,01:58:00,36759,17.269,True,Marketing


In [10]:
# Por fim, vemos que um dos tipos de pesquisa sempre será uma série booleana, assim como podemos definir uma coluna, caso passemos uma série booleana, uma comparação
is_female & is_marketing

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [11]:
over_salary = emp["Salary"] > 100000
emp[is_female & is_marketing & over_salary]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
98,Tina,Female,2016-06-16,07:47:00,100705,16.961,True,Marketing
140,Shirley,Female,1981-02-28,01:23:00,113850,1.854,False,Marketing
158,Norma,Female,1999-02-28,08:45:00,114412,8.756,True,Marketing
305,Margaret,Female,1993-02-06,01:05:00,125220,3.733,False,Marketing
319,Jacqueline,Female,1981-11-25,03:01:00,145988,18.243,False,Marketing
379,NaN,Female,2002-09-18,12:39:00,118906,4.537,True,Marketing
468,Janice,Female,1997-06-28,01:48:00,136032,10.696,True,Marketing
490,Judith,Female,2007-11-23,01:22:00,117055,7.461,False,Marketing
531,Virginia,Female,2010-05-02,09:10:00,123649,10.154,True,Marketing
585,Shirley,Female,1988-04-16,11:09:00,132156,2.754,False,Marketing


## Filter with More than One Condition (OR)
- Use the `|` operator in between two Boolean **Series** to filter by *either* condition.

In [12]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [13]:
# Seguiremos a lógica do or que já estamos acostumados, se um for verdadeiro, a comparação é verdadeira
is_senior_management = emp["Senior Management"] # Lembrando da redundância dos booleanos, pois o true já vale para a ordem original e false para o inverso quando se trata de listas em boolean
started_in_80s = emp["Start Date"] < "1990-01-01" # Lembrando que as formatações de data têm de coincidir, então temos de tomar cuidado para usar um formato e comparar neste mesmo formato ou equivalente
emp[is_senior_management | started_in_80s]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
...,...,...,...,...,...,...,...,...
992,Anthony,Male,2011-10-16,08:35:00,112769,11.625,True,Finance
993,Tina,Female,1997-05-15,03:53:00,56450,19.040,True,Engineering
994,George,Male,2013-06-21,05:47:00,98874,4.479,True,Marketing
996,Phillip,Male,1984-01-31,06:30:00,42392,19.675,False,Finance


In [14]:
Robert = emp["First Name"] == "Robert"
Client_Services = emp["Team"] == "Client Services"
Start = emp["Start Date"] > "2016-06-01"
emp[Robert & (Client_Services | Start)] # Assim como na álgebra, temos regras e prioridades, o python pode começar pelo or e depois ir pro and ou vice-versa, o que gera resultados diferentes, ou seja, devemos isolar e descrever com os parênteses quais serão primeiro e quais serão executados isolados para gerar o correto, de forma que sejam operados juntos para depois operar com os outros, mantendo as seleções (regra padrão da matemática)

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
387,Robert,Male,1994-10-29,04:26:00,123294,19.894,False,Client Services


## The isin Method
- The `isin` **Series** method accepts a collection object like a list, tuple, or **Series**.
- The method returns True for a row if its value is found in the collection.

In [15]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [16]:
# Mesma regra do in no SQL, pois a partir de vários valores, ele irá comparar e se um apenas for verdadeiro, a comparação será verdadeira
legal = emp["Team"] == "Legal"
sales = emp["Team"] == "Sales"
product = emp["Team"] == "Product"

emp[legal | sales | product]
emp[emp["Team"].isin(["Legal", "Sales", "Product"])] # É um método mais rápido do que múltiplos or, pois em uma or, and ou outras, ele faz uma comparação para cada item, ou seja, uma vez para legal, uma para sales e uma para product, já no isin ele faz apenas uma para o conjunto, reduzindo o número de processos
# Poderíamos passar uma lista, itens de um dicionário, uma série, em todos os casos de múltiplos valores o python entende como listagens e valores de itens separados, desde que a formatação obedeça (como a lista acima, que poderia estar em uma variável. Então podemos deixar o user decidir um dicionário ou nós mesmos e pesquisar
# Sempre devemos lembrar de designar a coluna que queremos procurar os valores, afinal, não vamos procurar no geral rsrs

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
6,Ruby,Female,1987-08-17,04:20:00,65476,10.012,True,Product
11,Julie,Female,1997-10-26,03:19:00,102508,12.637,True,Legal
13,Gary,Male,2008-01-27,11:40:00,109831,5.831,False,Sales
15,Lillian,Female,2016-06-05,06:09:00,59414,1.256,False,Product
...,...,...,...,...,...,...,...,...
981,James,Male,1993-01-15,05:19:00,148985,19.280,False,Legal
985,Stephen,NaN,1983-07-10,08:10:00,85668,1.909,False,Legal
989,Justin,NaN,1991-02-10,04:58:00,38344,3.794,False,Legal
997,Russell,Male,2013-05-20,12:39:00,96914,1.421,False,Product


## The isnull and notnull Methods
- The `isnull` method returns True for `NaN` values in a **Series**.
- The `notnull` method returns True for present values in a **Series**.

In [17]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [18]:
# Ambos são inversos um do outro, isnull vai dizer se há valores nulos (true) e notnull se há valores não nulos (true)
emp["Team"].isnull()
emp["Team"].notnull() #Vemos que ambos retornam uma s[erie booleana, ou seja, podem ser utilizados como parämetros de pesquisa

0       True
1      False
2       True
3       True
4       True
       ...  
995     True
996     True
997     True
998     True
999     True
Name: Team, Length: 1000, dtype: bool

In [19]:
# Podemos realizar diferentes pesquisas para trabalhar e tratar valores nulos, ou seja, limpar a base ou analisar, como ver nomes nulos mas que t[em times para trat[a-los
emp[emp["First Name"].isnull() & emp["Team"].notnull()]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
7,NaN,Female,2015-07-20,10:43:00,45906,11.598,True,Finance
25,NaN,Male,2012-10-08,01:12:00,37076,18.576,True,Client Services
39,NaN,Male,2016-01-29,02:33:00,122173,7.797,True,Client Services
51,NaN,NaN,2011-12-17,08:29:00,41126,14.009,True,Sales
62,NaN,Female,2007-06-12,05:25:00,58112,19.414,True,Marketing
116,NaN,Male,1991-06-22,08:58:00,76189,18.988,True,Legal
149,NaN,Female,2014-08-17,02:00:00,86230,8.578,True,Distribution
157,NaN,Female,2005-07-27,08:32:00,79536,14.443,True,Product
165,NaN,Female,2014-03-23,01:28:00,59148,9.061,True,Legal
166,NaN,Female,1991-07-09,06:52:00,42341,7.014,True,Sales


## The between Method
- The `between` method returns True if a **Series** value is found within its range.

In [20]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [21]:
# Assim como vemos em SQL, o between representa intervalo de valores
emp["Salary"].between(50000, 100000) # Retorna uma série booleana, assim como todos, que vai ser utilizada para a filtragem e combinação de valores, trazendo como vimos os valores em true (revisão)
emp[emp["Salary"].between(50000, 100000)]
emp[emp["Start Date"].between("1990-01-01", "1990-05-01")]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
52,Todd,Male,1990-02-18,02:41:00,49339,1.695,True,Human Resources
64,Kathleen,NaN,1990-04-11,06:46:00,77834,18.771,False,Business Development
251,Sharon,NaN,1990-03-01,03:46:00,83658,6.513,False,Business Development
310,Harold,Male,1990-02-20,11:00:00,66775,2.158,True,Legal
356,Judy,Female,1990-02-01,03:32:00,38092,5.668,False,Distribution
581,Ernest,Male,1990-01-28,12:08:00,81919,15.118,False,Marketing
632,Rebecca,Female,1990-02-23,03:21:00,134673,6.878,False,Engineering


In [22]:
import datetime as dt # Para trabalharmos com comparações de horas no formato dt.time(horas, minutos), que é uma das maneiras de interpretação e é como convertemos para ele entender as horas, então é como o sistema vai compreender
emp["Last Login Time"].between(dt.time(8, 30), dt.time(9, 0))
emp[emp["Last Login Time"].between(dt.time(8, 30), dt.time(9, 0))]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
36,Rachel,Female,2009-02-16,08:47:00,142032,12.599,False,Business Development
37,Linda,Female,1981-10-19,08:49:00,57427,9.557,True,Client Services
85,Jeremy,Male,2008-02-01,08:50:00,100238,3.887,True,Client Services
96,Cynthia,Female,1994-03-21,08:34:00,142321,1.737,False,Finance
116,NaN,Male,1991-06-22,08:58:00,76189,18.988,True,Legal
121,Kathleen,NaN,2016-05-09,08:55:00,119735,18.740,False,Product
141,Adam,Male,1990-12-24,08:57:00,110194,14.727,True,Product
150,Sean,Male,1996-05-04,08:59:00,135490,19.934,False,Marketing
153,Victor,NaN,2011-03-10,08:40:00,84546,10.489,True,Finance
157,NaN,Female,2005-07-27,08:32:00,79536,14.443,True,Product


## The duplicated Method
- The `duplicated` method returns True if a **Series** value is a duplicate.
- Pandas will mark one occurrence of a repeated value as a non-duplicate.
- Use the `keep` parameter to designate whether the first or last occurrence of a repeated value should be considered the "non-duplicate".
- Pass False to the `keep` parameter to mark all occurrences of repeated values as duplicates.
- Use the tilde symbol (`~`) to invert a **Series's** values. Trues will become Falses, and Falses will become trues.

In [23]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [24]:
# No Python, as duplicadas acontecem de maneira diferente, pois será montada uma lista booleana com os valores, contudo, a primeira ocorrência do valor (um nome por exemplo) nunca será duplicada, pois é a primeira vez que aparece, já as outras vão ser duplicadas, pois representam uma cópia, então são a duplicidade daquela primeira, ou seja, a que ele vê primeiro seria a original e ele marca como a duplicada as "falsas"
emp[emp["First Name"].duplicated()] # Logo, aqui não vemos as primeiras ocorrências, mas sim as duplicidades, as outras vezes que o valor aparece

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
23,NaN,Male,2012-06-14,04:19:00,125792,5.042,True,NaN
25,NaN,Male,2012-10-08,01:12:00,37076,18.576,True,Client Services
32,NaN,Male,1998-08-21,02:27:00,122340,6.417,True,NaN
34,Jerry,Male,2004-01-10,12:56:00,95734,19.096,False,Client Services
39,NaN,Male,2016-01-29,02:33:00,122173,7.797,True,Client Services
...,...,...,...,...,...,...,...,...
995,Henry,NaN,2014-11-23,06:09:00,132483,16.655,False,Distribution
996,Phillip,Male,1984-01-31,06:30:00,42392,19.675,False,Finance
997,Russell,Male,2013-05-20,12:39:00,96914,1.421,False,Product
998,Larry,Male,2013-04-20,04:45:00,60500,11.985,False,Business Development


In [25]:
# Podemos utilizar o parâmetro keep para definir qual será o valor mantido, ou seja, o valor original que não será marcado como duplicado
emp[emp["First Name"].duplicated(keep = "last")] # Aqui vemos outros dados, pois manteve o último e os anteriores foram os duplicados

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services
...,...,...,...,...,...,...,...,...
959,Albert,Male,1992-09-19,02:35:00,45094,5.850,True,Business Development
960,Stephen,Male,1989-10-29,11:34:00,93997,18.093,True,Business Development
970,Alice,Female,1988-09-03,08:54:00,63571,15.397,True,Product
973,Russell,Male,2013-05-10,11:08:00,137359,11.105,False,Business Development


In [26]:
# Do mesmo modo, podemos fazer com que ele não retire da listagem este parâmetro original, fazendo com que o keep seja desligado e todos sejam retornados, pois em tese ele vai rodar o first e last
emp[emp["First Name"].duplicated(keep=False)] # Logo, quando utilizamos este parâmetro, podemos sempre nos debruçar sobre os valores abaixo, para saber quantos registros foram retornados, ou então fazer uma função de retorno para ver quantos foram, porém, abaixo podemos ver as 991 linhas que foram retornadas, ou seja, se temos 1000 itens, apenas 8 serão únicos. Podemos sempre utilizar estas informações para nos basear, saber a quantidade de linhas, retornos ou dados filtrados

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services
...,...,...,...,...,...,...,...,...
995,Henry,NaN,2014-11-23,06:09:00,132483,16.655,False,Distribution
996,Phillip,Male,1984-01-31,06:30:00,42392,19.675,False,Finance
997,Russell,Male,2013-05-20,12:39:00,96914,1.421,False,Product
998,Larry,Male,2013-04-20,04:45:00,60500,11.985,False,Business Development


In [27]:
# Para revertermos então a série booleana, isso para quaisquer consultas, o que pode nos ajudar, já que aqui colocando o inverso das duplicadas, saberemos quais são únicas, utilizamos o ~
# Utilizando o símbolo, podemos reverter a série, o que era true (os duplicados) agora será true (os únicos)
emp[~emp["First Name"].duplicated(keep=False)]

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
8,Angela,Female,2005-11-22,06:29:00,95570,18.523,True,Engineering
33,Jean,Female,1993-12-18,09:07:00,119082,16.180,False,Business Development
190,Carol,Female,1996-03-19,03:39:00,57783,9.129,False,Finance
291,Tammy,Female,1984-11-11,10:30:00,132839,17.463,True,Client Services
495,Eugene,Male,1984-05-24,10:54:00,81077,2.117,False,Sales
688,Brian,Male,2007-04-07,10:47:00,93901,17.821,True,Legal
832,Keith,Male,2003-02-12,03:02:00,120672,19.467,False,Legal
887,David,Male,2009-12-05,08:48:00,92242,15.407,False,Legal


## The drop_duplicates Method
- The `drop_duplicates` method deletes rows with duplicate values.
- By default, it will remove a row if *all* of its values are shared with another row.
- The `subset` parameter configures the columns to look for duplicate values within.
- Pass a list to `subset` parameter to look for duplicates across multiple columns.

In [28]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [29]:
# Sabemos que o python compreende valores duplicados como o conjunto que estamos considerando, ou seja, caso consideremos o parâmetro sem nada, ele não terá nenhum registro igual, pois queremos saber se o registro completo está duplicado (igual ao SQL)
emp.drop_duplicates("Team") # Vemos aqui que é semelhante ao parâmetro duplicate, ele vai trazer apenas os primeiros valores que aparecem, assim como um distinct que será feito apenas naquela coluna, ou seja, trará apenas um exemplo de cada, apenas a primeira ocorrência de cada e dispensará os duplicados, trazendo apenas os distintintos valores, como um sumarize

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
6,Ruby,Female,1987-08-17,04:20:00,65476,10.012,True,Product
8,Angela,Female,2005-11-22,06:29:00,95570,18.523,True,Engineering
9,Frances,Female,2002-08-08,06:51:00,139852,7.524,True,Business Development
12,Brandon,Male,1980-12-01,01:08:00,112807,17.492,True,Human Resources
13,Gary,Male,2008-01-27,11:40:00,109831,5.831,False,Sales


In [34]:
# Podemos utilizar outros parâmetros como o keep, lembrando que ele vai trazer as linhas do fim ou do ínicio, mantendo elas e trazendo as distintas de acordo com as primeiras que encontrar (semelhante ao SQL) e se colocarmos false, trará apenas os valores que aparecem uma vez, ou seja, que não tiveram duplicados, o que no caso abaixo gera uma lista vazia pois todos os valores constam como duplicados, ou seja, o keep false não atende
emp.drop_duplicates("Team", keep="last")
# Sempre lembrar que o false no keep apenas diz o que vai manter, e como não temos um específico, ele considera na série todos, logo, se for para excluir os duplicados, exclui todos, já se for para selecionar, seleciona todos que são duplicados
# Podemos também ver quais são unicos com o parâmetro keep false, já que ele vai rodar first e last e não achar outros, ou seja, não terá duplicados dele (se necessário, revisar lógica)
emp.drop_duplicates("First Name", keep=False)

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
8,Angela,Female,2005-11-22,06:29:00,95570,18.523,True,Engineering
33,Jean,Female,1993-12-18,09:07:00,119082,16.180,False,Business Development
190,Carol,Female,1996-03-19,03:39:00,57783,9.129,False,Finance
291,Tammy,Female,1984-11-11,10:30:00,132839,17.463,True,Client Services
495,Eugene,Male,1984-05-24,10:54:00,81077,2.117,False,Sales
688,Brian,Male,2007-04-07,10:47:00,93901,17.821,True,Legal
832,Keith,Male,2003-02-12,03:02:00,120672,19.467,False,Legal
887,David,Male,2009-12-05,08:48:00,92242,15.407,False,Legal


In [35]:
# Podemos então definir quais linhas serão combinadas para definir os duplicados
emp.drop_duplicates(["Senior Management", "Team"], keep="first").sort_values("Team", ascending=False) # Exemplos utilizando múltiplos parâmetros

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
13,Gary,Male,2008-01-27,11:40:00,109831,5.831,False,Sales
45,Roger,Male,1980-04-17,11:32:00,88010,13.886,True,Sales
6,Ruby,Female,1987-08-17,04:20:00,65476,10.012,True,Product
15,Lillian,Female,2016-06-05,06:09:00,59414,1.256,False,Product
21,Matthew,Male,1995-09-05,02:12:00,100612,13.645,False,Marketing
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
11,Julie,Female,1997-10-26,03:19:00,102508,12.637,True,Legal
5,Dennis,Male,1987-04-18,01:35:00,115163,10.125,False,Legal
12,Brandon,Male,1980-12-01,01:08:00,112807,17.492,True,Human Resources
16,Jeremy,Male,2010-09-21,05:56:00,90370,7.369,False,Human Resources


## The unique and nunique Methods
- The `unique` method on a **Series** returns a collection of its unique values. The method does not exist on a **DataFrame**.
- The `nunique` method returns a *count* of the number of unique values in the **Series**/**DataFrame**.
- The `dropna` parameter configures whether to include or exclude missing (`NaN`) values.

In [37]:
emp = pd.read_csv("employees.csv", parse_dates=["Start Date"], date_format="%m/%d/%Y")
emp["Last Login Time"] = pd.to_datetime(emp["Last Login Time"], format="%H:%M %p").dt.time
emp["Senior Management"] = emp["Senior Management"].astype(bool)
emp["Gender"] = emp["Gender"].astype("category")
emp.head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,1993-08-06,12:42:00,97308,6.945,True,Marketing
1,Thomas,Male,1996-03-31,06:53:00,61933,4.170,True,NaN
2,Maria,Female,1993-04-23,11:17:00,130590,11.858,False,Finance
3,Jerry,Male,2005-03-04,01:00:00,138705,9.340,True,Finance
4,Larry,Male,1998-01-24,04:47:00,101004,1.389,True,Client Services


In [39]:
# Podemos utilizar parâmetros unique e nunique (quantidade de únicos) para saber quais valores são únicos, lembrando que ele só funciona com uma série, ou seja, uma coluna. podemos saber então quais são únicos e quais são diferentes sem puxar os registros para conseguirmos contar valores únicos, combinar parâmetros ou retirar informações do banco
# O parâmetro adequa o resultado a um tipo específico para ele, ou seja, se for de muita repetição, vai ser object e caso se repita pouco, por ordem de processamento, converte para categoria
emp["Gender"].unique()
type(emp["Gender"].unique())

pandas.core.arrays.categorical.Categorical

In [ ]:
emp["Team"].unique()
type(emp["Team"].unique())

In [41]:
emp["Team"].nunique()

10

In [44]:
# Podemos dropar os valores nulos, sem considerar na conta, sabendo quais os valores válidos diferentes, quais existem fora o nan e são únicos (a soma é apresentada)
emp["Team"].nunique(dropna=True) # Este parâmetro é  padrão, podemos então considerar o nan, fazendo com que apresente os únicos e se houver nan, inclua ele também como um valor único ou não


10

In [47]:
# Podemos utilizar o nunique de maneira geral, para saber quantos valores únicos temos, lembrando que ele separa as colunas, ou seja, o unique trabalha com as colunas e o nunique pode trabalhar com o dataframe mas operará cada coluna
emp.nunique() # Isso não funciona com o unique porque ele trabalha com séries, não com o conjunto

First Name           200
Gender                 2
Start Date           972
Last Login Time      542
Salary               995
Bonus %              971
Senior Management      2
Team                  10
dtype: int64